In [1]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.models import Model
import numpy as np 
import pandas as pd 
import re
import nltk
from preprocess import *
from models import *

2022-12-02 20:56:28.403071: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 20:56:30.401887: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2022-12-02 20:56:30.402185: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2022-12-02 20:56:30.402198: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] 

In [2]:
df = pd.read_csv("questions.csv")
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
df.shape

(404351, 6)

In [4]:
#pick a sample of 5000 distinct random rows
df = df.sample(n=100000, random_state=1)
df.shape

(100000, 6)

In [5]:
# question_1, question_2 = df['question1'].to_list(), df['question2'].to_list()
# is_duplicate = df['is_duplicate'].to_list()
# preprocess_neural(question_1, question_2, is_duplicate)

In [6]:
q1_inp, q2_inp, is_duplicate = df['question1'].to_list(), df['question2'].to_list(), df['is_duplicate'].to_list()

Acquired Test data

In [7]:
MAX_WORDS_VOCAB = 200000
tokenizer = Tokenizer(num_words = MAX_WORDS_VOCAB, lower=False, split=" ")
tokenizer.fit_on_texts(list(df['question1'].values.astype(str))+list(df['question2'].values.astype(str)))

In [8]:
print("Number of words in vocabulary: ", len(tokenizer.word_index))

Number of words in vocabulary:  60949


In [9]:

q1_sequence = tokenizer.texts_to_sequences(df['question1'].values.astype(str))
q1_sequence = pad_sequences(q1_sequence, maxlen = 128)

q2_sequence = tokenizer.texts_to_sequences(df['question2'].values.astype(str))
q2_sequence = pad_sequences(q2_sequence, maxlen = 128)

In [10]:
windex = tokenizer.word_index

In [11]:
embedding_index = {}
with open('glove.6B.300d.txt','r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], 'float32')
        embedding_index[word] = vectors
    f.close()

In [12]:
embedding_matrix = np.random.random((len(windex)+1, 300))

for word, i in windex.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)

(60950, 300)


In [13]:
print(len(windex)+1)

60950


In [14]:
#split the data into 70-20-10 train-validation-test with random state 42
from sklearn.model_selection import train_test_split
q1_train, q1_test, q2_train, q2_test, y_train, y_test = train_test_split(q1_sequence, q2_sequence, is_duplicate, test_size=0.1, random_state=42)
q1_train, q1_val, q2_train, q2_val, y_train, y_val = train_test_split(q1_train, q2_train, y_train, test_size=0.2, random_state=42)

In [15]:
import tensorflow as tf
y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=2)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=2)

In [16]:
is_duplicate = np.array(is_duplicate)
print(is_duplicate.shape)

(100000,)


In [17]:
#print the ratio of positive and negative samples in train, validation and test
y_train, y_val, y_test = np.array(y_train), np.array(y_val), np.array(y_test)
print("Train: ", sum(y_train)/len(y_train))
print("Validation: ", sum(y_val)/len(y_val))
print("Test: ", sum(y_test)/len(y_test))

Train:  [0.63294444 0.36705556]
Validation:  [0.6358333  0.36416668]
Test:  [0.6291 0.3709]


In [107]:
# %load models.py
from keras.layers import *
from keras.models import Model
import tensorflow as tf
import numpy as np 
from keras import Sequential
from keras.optimizers import adam_v2
import keras as K

class NeuralModels:
    def __init__(self, emb_mat, vocab_size = -1, loss = "binary_crossentropy", epochs = 10, optimizer = "adam", metrics = ["accuracy"]):
        self.epochs = epochs
        self.optimizer = optimizer
        self.loss = loss
        self.metrics = metrics
        self.vocab_size = vocab_size
        self.embedding_matrix = emb_mat
        self.model = Sequential()

class CBOW(NeuralModels):
    def __init__(self, emb_mat, vocab_size = -1, loss = "binary_crossentropy", epochs = 10, optimizer = "adam", metrics = ["accuracy"]):
        super().__init__(emb_mat, vocab_size, loss, epochs, optimizer, metrics)
    
    def fit(self,xtrain, xval, ytrain, yval):
        self.model.add(Dense(300, input_shape = (900,), activation = "sigmoid"))
        self.model.add(Dropout(0.1))
        self.model.add(Dense(200, activation = "sigmoid"))
        self.model.add(Dropout(0.1))
        self.model.add(Dense(100, activation = "sigmoid"))
        self.model.add(Dropout(0.1))
        self.model.add(Dense(2, activation = "softmax"))
        self.model.compile(loss = self.loss, optimizer = self.optimizer, metrics = self.metrics)
        self.model.fit(xtrain, ytrain, epochs = self.epochs, validation_data = (xval, yval), batch_size = 64, verbose = 1)

    def get_model_summary(self):
        self.model.summary()
    
    def predict(self, xtest):
        return self.model.predict(xtest)

class LsTM(NeuralModels):
    def __init__(self, emb_mat, vocab_size = -1, loss = "binary_crossentropy", epochs = 10, optimizer = "adam", metrics = ["accuracy"]):
        super().__init__(emb_mat, vocab_size, loss, epochs, optimizer, metrics)
    
    def train_model(self):
        inp1 = Input(shape = (128,))
        inp2 = Input(shape = (128,))
        emb1 = Embedding(output_dim=300, weights = [self.embedding_matrix], trainable = False, input_dim=self.vocab_size, input_length=128)(inp1)
        emb2 = Embedding(output_dim=300, weights = [self.embedding_matrix], trainable = False, input_dim=self.vocab_size, input_length=128)(inp2)
        concat = Concatenate(axis = -1)([emb1 + emb2, emb1 - emb2, emb1 * emb2])
        lstm = LSTM(150, return_sequences=False, dropout=0.2, return_state=True)(concat)
        out = Dense(2, activation = "softmax")(lstm[2])
        self.model = Model(inputs = [inp1, inp2], outputs = out)
        self.model.compile(loss = self.loss, optimizer = adam_v2.Adam(learning_rate=0.0008), metrics = self.metrics)

    def get_model_summary(self):
        self.model.summary()
    
    def predict(self, xtest):
        return self.model.predict(xtest)

class BiLSTM(NeuralModels):
    def __init__(self, emb_mat, vocab_size = -1, loss = "binary_crossentropy", epochs = 10, optimizer = "adam", metrics = ["accuracy"]):
        super().__init__(emb_mat, vocab_size, loss, epochs, optimizer, metrics)
    
    def train_model(self):
        inp1 = Input(shape = (128,))
        inp2 = Input(shape = (128,))
        emb1 = Embedding(output_dim=300, weights = [self.embedding_matrix], trainable = False, input_dim=self.vocab_size, input_length=128)(inp1)
        emb2 = Embedding(output_dim=300, weights = [self.embedding_matrix], trainable = False, input_dim=self.vocab_size, input_length=128)(inp2)
        concat = Concatenate(axis = -1)([emb1 + emb2, emb1 - emb2, emb1 * emb2])
        out = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150, kernel_regularizer='l2', dropout=0.1, return_sequences=True))(concat)
        out = tf.keras.backend.mean(out, axis=1, keepdims=False)
        output = tf.keras.layers.Dense(2, kernel_regularizer='l2', activation='softmax')(out)
        self.model = Model(inputs = [inp1, inp2], outputs = output)
        self.model.compile(loss = self.loss, optimizer = self.optimizer, metrics = self.metrics)

    def get_model_summary(self):
        self.model.summary()
    
    def predict(self, xtest):
        return self.model.predict(xtest)

class LsTM_Attention(NeuralModels):
    def __init__(self, emb_mat, vocab_size = -1, loss = "binary_crossentropy", epochs = 10, optimizer = "adam", metrics = ["accuracy"]):
        super().__init__(emb_mat, vocab_size, loss, epochs, optimizer, metrics)
    
    def train_model(self):
        inp1 = Input(shape = (128,))
        inp2 = Input(shape = (128,))
        emb1 = Embedding(output_dim=300, weights = [self.embedding_matrix], trainable = False, input_dim=self.vocab_size, input_length=128)(inp1)
        emb2 = Embedding(output_dim=300, weights = [self.embedding_matrix], trainable = False, input_dim=self.vocab_size, input_length=128)(inp2)
        
        lstm1 = LSTM(150, return_sequences=True, dropout=0.1, return_state=True)(emb1)
        lstm2 = LSTM(150, return_sequences=True, dropout=0.1, return_state=True)(emb2)

        attention = dot([lstm1[0], lstm2[0]], axes=[2, 2])
        u_norm = Softmax(axis=-1)(attention)
        v_norm = Softmax(axis=1)(attention)


        u = dot([u_norm, lstm1[0]], axes=[1, 1])
        v = dot([v_norm, lstm2[0]], axes=[1, 1])

        WU_bar = Dense(150)(u[:, -1, :])
        WV_bar = Dense(150)(v[:, -1, :])
        VU = Dense(150)(lstm1[0][:, -1, :])
        VV = Dense(150)(lstm2[0][:, -1, :])

        ufinal = Add()([WU_bar, VU])
        vfinal = Add()([WV_bar, VV])

        ufinal = Activation('tanh')(ufinal)
        vfinal = Activation('tanh')(vfinal)
        
        concat = Concatenate(axis = -1)([ufinal, vfinal])
        out = Dense(2, activation = "softmax")(concat)
        self.model = Model(inputs = [inp1, inp2], outputs = out)
        self.model.compile(loss = self.loss, optimizer = self.optimizer, metrics = self.metrics)


    def get_model_summary(self):
        self.model.summary()
    
    def predict(self, xtest):
        return self.model.predict(xtest)

In [108]:
model = LsTM_Attention(embedding_matrix, len(windex) + 1, loss="categorical_crossentropy")

In [109]:
model.train_model()

In [110]:
model.get_model_summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_45 (InputLayer)          [(None, 128)]        0           []                               
                                                                                                  
 input_46 (InputLayer)          [(None, 128)]        0           []                               
                                                                                                  
 embedding_44 (Embedding)       (None, 128, 300)     18285000    ['input_45[0][0]']               
                                                                                                  
 embedding_45 (Embedding)       (None, 128, 300)     18285000    ['input_46[0][0]']               
                                                                                            

In [111]:
print(q1_val.shape)

(18000, 128)


In [112]:
model.model.fit([q1_train, q2_train], y_train, epochs = 4, validation_data = ([q1_val, q2_val], y_val), batch_size = 8, validation_batch_size=4, verbose = 1)

Epoch 1/4
9000/9000 [==============================] - 1253s 139ms/step - loss: 0.5809 - accuracy: 0.6902 - val_loss: 0.5423 - val_accuracy: 0.7200
Epoch 2/4
9000/9000 [==============================] - 1242s 138ms/step - loss: 0.5190 - accuracy: 0.7422 - val_loss: 0.5348 - val_accuracy: 0.7373
Epoch 3/4
9000/9000 [==============================] - 1238s 138ms/step - loss: 0.4668 - accuracy: 0.7751 - val_loss: 0.5124 - val_accuracy: 0.7573
Epoch 4/4
9000/9000 [==============================] - 1248s 139ms/step - loss: 0.4030 - accuracy: 0.8133 - val_loss: 0.4988 - val_accuracy: 0.7642


In [113]:
y_pred = model.predict([q1_test, q2_test])
y_pred1d, y_actual1d = [], []
for i in range(len(y_test)):
    if(y_test[i][0] == 1):
        y_actual1d.append(0)
    else:
        y_actual1d.append(1)

for i in range(len(y_pred)):
    if(y_pred[i][0] > y_pred[i][1]):
        y_pred1d.append(0)
    else:
        y_pred1d.append(1)

from sklearn.metrics import accuracy_score, f1_score
print("Accuracy: ", accuracy_score(y_actual1d, y_pred1d))
print("F1 Score: ", f1_score(y_actual1d, y_pred1d))

313/313 [==============================] - 31s 87ms/step
Accuracy:  0.7751
F1 Score:  0.7064308681672025
